# Import

In [66]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [67]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [68]:
# policy year level data
df = pd.read_csv('/content/drive/MyDrive/BU Capstone Team2 Credit No_hit/Data/team2_driver.csv.gz')

In [69]:
df.head()

,pol_id,year,STATE,ZIP5,ZIP4,no_hit
0,BHD00001001024,2016,MA,2740,1228.0,1
1,BHD00001001024,2017,MA,2740,1228.0,1
2,BHD00001001024,2018,MA,2740,1228.0,1
3,BHD00001001024,2019,MA,2740,1228.0,1
4,BHD00001001024,2020,MA,2740,1228.0,1


In [70]:
# id level data
pol = pd.read_csv('/content/drive/MyDrive/BU Capstone Team2 Credit No_hit/Data/policy_level.csv').drop('Unnamed: 0', axis=1)

In [71]:
pol.head()

,pol_id,year,STATE,ZIP5,ZIP4,no_hit
0,BHD00001001024,2016,MA,2740,1228.0,1
1,BHD00001001030,2016,MA,2740,4523.0,1
2,BHD00001001034,2015,MA,2745,5672.0,0
3,BHD00001001037,2016,MA,1566,1335.0,0
4,BHD00001001041,2016,MA,2745,5260.0,0


In [72]:
pol.shape

(176704, 6)

# useful stratify functions
https://www.kaggle.com/flaviobossolan/stratified-sampling-python

In [73]:
def stratified_sample(df, strata, size=None, seed=None, keep_index= True):
  population = len(df)
  size = __smpl_size(population, size)
  tmp = df[strata]
  tmp['size'] = 1
  tmp_grpd = tmp.groupby(strata).count().reset_index()
  tmp_grpd['samp_size'] = round(size/population * tmp_grpd['size']).astype(int)
  first = True 
  for i in range(len(tmp_grpd)):
        # query generator for each iteration
    qry=''
    for s in range(len(strata)):
      stratum = strata[s]
      value = tmp_grpd.iloc[i][stratum]
      n = tmp_grpd.iloc[i]['samp_size']

      if type(value) == str:
        value = "'" + str(value) + "'"
            
      if s != len(strata)-1:
        qry = qry + stratum + ' == ' + str(value) +' & '

      else:
        qry = qry + stratum + ' == ' + str(value)
        
        
      if first:
        stratified_df = df.query(qry).sample(n=n, random_state=seed).reset_index(drop=(not keep_index))
        first = False
      else:
        tmp_df = df.query(qry).sample(n=n, random_state=seed).reset_index(drop=(not keep_index))
        stratified_df = stratified_df.append(tmp_df, ignore_index=True)
    
    return stratified_df


In [74]:
def stratified_sample_report(df, strata, size=None):
    population = len(df)
    size = __smpl_size(population, size)
    tmp = df[strata]
    tmp['size'] = 1
    tmp_grpd = tmp.groupby(strata).count().reset_index()
    tmp_grpd['samp_size'] = round(size/population * tmp_grpd['size']).astype(int)
    return tmp_grpd


def __smpl_size(population, size):
    if size is None:
        cochran_n = round(((1.96)**2 * 0.5 * 0.5)/ 0.02**2)
        n = round(cochran_n/(1+((cochran_n -1) /population)))
    elif size >= 0 and size < 1:
        n = round(population * size)
    elif size < 0:
        raise ValueError('Parameter "size" must be an integer or a proportion between 0 and 0.99.')
    elif size >= 1:
        n = size
    return n


# Policy id (in sample)

In [75]:
p_in = pol[pol.year.isin([2016, 2017, 2018, 2019])]
p_in

,pol_id,year,STATE,ZIP5,ZIP4,no_hit
0,BHD00001001024,2016,MA,2740,1228.0,1
1,BHD00001001030,2016,MA,2740,4523.0,1
3,BHD00001001037,2016,MA,1566,1335.0,0
4,BHD00001001041,2016,MA,2745,5260.0,0
5,BHD00001001042,2016,MA,2740,4636.0,1
...,...,...,...,...,...,...
176699,PNH00002027385,2018,PA,18322,7702.0,1
176700,PNH00002027689,2018,PA,18353,7961.0,0
176701,PNH00002028175,2018,NJ,18350,NaN,1
176702,PNH00002028506,2018,PA,18052,4002.0,0


In [76]:
p_in.loc[((p_in['STATE']=='NY')& (p_in['year']==2018)), 'year']=2019

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [77]:
p_in.groupby('STATE')['pol_id'].count()

STATE
CT    10109
MA    55380
NH     6613
NJ    41017
NY     1338
PA    17456
Name: pol_id, dtype: int64

In [78]:
p_in.groupby('year')['pol_id'].count()

year
2016    38605
2017    13133
2018    34959
2019    45216
Name: pol_id, dtype: int64

In [79]:
p_in.groupby('no_hit')['pol_id'].count()

no_hit
0    123837
1      8076
Name: pol_id, dtype: int64

In [80]:
p_in.groupby(['STATE','year','no_hit'])['pol_id'].count()

STATE  year  no_hit
CT     2016  0          3250
             1            91
       2017  0          1989
             1            75
       2018  0          1654
             1            73
       2019  0          2626
             1           351
MA     2016  0         28485
             1          1016
       2017  0          6208
             1           211
       2018  0          8187
             1           243
       2019  0         10309
             1           721
NH     2016  0          1850
             1           227
       2017  0          1071
             1            83
       2018  0          1242
             1           129
       2019  0          1733
             1           278
NJ     2016  0          3518
             1           168
       2017  0          3161
             1           180
       2018  0         17557
             1           643
       2019  0         14536
             1          1254
NY     2019  0          1145
             1         

In [81]:
p_in_X = p_in.drop('no_hit', axis=1)
p_in_y = p_in[['no_hit']]

In [82]:
# train test split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(p_in_X, p_in_y, test_size=0.2, random_state=2, stratify = p_in[['STATE','year','no_hit']])

> **in-sample training set**

In [83]:
X_train['y'] = y_train
X_train

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,pol_id,year,STATE,ZIP5,ZIP4,y
147024,PNH00002007336,2018,PA,19021,7538.0,0
134232,PAH00001632657,2018,NJ,8003,1522.0,0
30478,BHH00001072913,2017,MA,2346,2123.0,0
1251,BHD10001498264,2016,MA,1545,5021.0,1
168025,BHH00001094401,2018,MA,1469,1420.0,0
...,...,...,...,...,...,...
94549,NJH00002007620,2019,NJ,8088,1202.0,0
128591,PAH00001509164,2018,NJ,7950,3424.0,0
26562,BHH00001063822,2016,MA,2482,2263.0,0
138565,PAH00001720143,2019,NJ,8080,1974.0,0


In [84]:
X_train.groupby(['year','y'])['pol_id'].count()

year  y
2016  0    29682
      1     1202
2017  0    10063
      1      443
2018  0    26661
      1     1307
2019  0    32664
      1     3508
Name: pol_id, dtype: int64

In [85]:
X_train.groupby('year')['pol_id'].count()

year
2016    30884
2017    10506
2018    27968
2019    36172
Name: pol_id, dtype: int64

In [86]:
X_train.groupby('STATE')['pol_id'].count()

STATE
CT     8088
MA    44304
NH     5290
NJ    32813
NY     1070
PA    13965
Name: pol_id, dtype: int64

In [87]:
y_train.value_counts()

no_hit
0         99070
1          6460
dtype: int64

In [88]:
X_train.to_csv('id_Xtrain.csv')

 > **in-sample testing set**

In [89]:
X_test

,pol_id,year,STATE,ZIP5,ZIP4
153113,PNH00002016383,2019,PA,19380,1336.0
145842,PNH00002005641,2018,PA,19149,1933.0
157032,PNH00002021880,2019,PA,17870,7659.0
58292,BHH10001340552,2016,MA,1342,9704.0
40770,BHH00001095029,2018,MA,1801,4352.0
...,...,...,...,...,...
150155,PNH00002011877,2019,PA,19121,2912.0
133739,PAH00001621989,2019,NJ,8060,3269.0
152555,PNH00002015645,2019,PA,19082,3310.0
84264,MWH00001014225,2016,NH,3827,3526.0


In [90]:
X_test['y'] = y_test
X_test.groupby(['year','y'])['pol_id'].count()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


year  y
2016  0    7421
      1     300
2017  0    2516
      1     111
2018  0    6664
      1     327
2019  0    8166
      1     878
Name: pol_id, dtype: int64

In [91]:
X_test.groupby('year')['pol_id'].count()

year
2016    7721
2017    2627
2018    6991
2019    9044
Name: pol_id, dtype: int64

In [92]:
X_test.groupby('STATE')['pol_id'].count()

STATE
CT     2021
MA    11076
NH     1323
NJ     8204
NY      268
PA     3491
Name: pol_id, dtype: int64

In [93]:
y_test.value_counts()

no_hit
0         24767
1          1616
dtype: int64

In [94]:
X_test.to_csv('id_Xtest.csv')

# Policy id (out of sample)

In [95]:
p_out = pol[pol.year.isin([2011, 2012, 2013, 2014, 2015, 2020])]
p_out

,pol_id,year,STATE,ZIP5,ZIP4,no_hit
2,BHD00001001034,2015,MA,2745,5672.0,0
6,BHD00001001080,2015,MA,1850,2164.0,0
18,BHD00001001157,2015,MA,2122,2204.0,0
20,BHD00001001189,2015,MA,1571,3437.0,1
25,BHD00001001232,2015,MA,2151,5005.0,0
...,...,...,...,...,...,...
173116,PAH00001348301,2015,NJ,7606,1405.0,0
173137,PAH00001371633,2015,NJ,7746,2106.0,0
173145,PAH00001377970,2015,NJ,7012,1932.0,0
173158,PAH00001379990,2015,NJ,8046,3511.0,0


In [96]:
p_out.to_csv('id_outsample.csv')

In [97]:
p_out_X = p_out.drop('no_hit', axis=1)
p_out_y = p_out[['no_hit']]

In [98]:
p_out_X.groupby('year')['pol_id'].count()

year
2011     3395
2012      767
2013     2409
2014     3045
2015     8441
2020    26734
Name: pol_id, dtype: int64

In [99]:
p_out_X.groupby('STATE')['pol_id'].count()

STATE
CT     9413
MA    11911
NH     7060
NJ     6527
NY     5760
PA     4120
Name: pol_id, dtype: int64

In [100]:
p_out_y.value_counts()

no_hit
0         36974
1          7817
dtype: int64

In [101]:
p_out.groupby(['STATE','no_hit'])['pol_id'].count()

STATE  no_hit
CT     0          8096
       1          1317
MA     0         10123
       1          1788
NH     0          6433
       1           627
NJ     0          5033
       1          1494
NY     0          4624
       1          1136
PA     0          2665
       1          1455
Name: pol_id, dtype: int64

# Policy id + year (in sample)

> **in-sample training set**

In [102]:
pol_train_id = X_train[['pol_id']]

In [103]:
idyr = pd.merge(df, pol_train_id, how='inner', on='pol_id')
idyr

,pol_id,year,STATE,ZIP5,ZIP4,no_hit
0,BHD00001001024,2016,MA,2740,1228.0,1
1,BHD00001001024,2017,MA,2740,1228.0,1
2,BHD00001001024,2018,MA,2740,1228.0,1
3,BHD00001001024,2019,MA,2740,1228.0,1
4,BHD00001001024,2020,MA,2740,1228.0,1
...,...,...,...,...,...,...
241797,PNH00002031580,2018,PA,16428,1003.0,1
241798,PNH00002031580,2019,PA,16428,1003.0,1
241799,PNH00002031723,2019,PA,18042,6377.0,0
241800,PNH00002031929,2018,PA,19007,4444.0,0


In [104]:
idyr_X_train = idyr.drop('no_hit', axis=1)
idyr_y_train = idyr[['no_hit']]

In [105]:
idyr.groupby(['year','no_hit'])['pol_id'].count()

year  no_hit
2016  0         27956
      1          2928
2017  0         33597
      1          3563
2018  0         54855
      1          5725
2019  0         82164
      1          7701
2020  0         21769
      1          1544
Name: pol_id, dtype: int64

In [106]:
idyr_X_train.groupby('year')['pol_id'].count()

year
2016    30884
2017    37160
2018    60580
2019    89865
2020    23313
Name: pol_id, dtype: int64

In [107]:
idyr_X_train.groupby('STATE')['pol_id'].count()

STATE
CT     18519
MA    127009
NH     12444
NJ     60442
NY      1237
PA     22151
Name: pol_id, dtype: int64

In [108]:
idyr_y_train.value_counts()

no_hit
0         220341
1          21461
dtype: int64

In [109]:
idyr.to_csv('idyr_train.csv')

> **in-sample testing set**

In [110]:
pol_test_id = X_test[['pol_id']]

In [111]:
idyr_t = pd.merge(df, pol_test_id, how='inner', on='pol_id')
idyr_t

,pol_id,year,STATE,ZIP5,ZIP4,no_hit
0,BHD00001001088,2016,MA,1570,2246.0,0
1,BHD00001001088,2017,MA,1570,2246.0,0
2,BHD00001001088,2018,MA,1570,2246.0,0
3,BHD00001001136,2016,MA,1570,3637.0,1
4,BHD00001001136,2017,MA,1570,3637.0,1
...,...,...,...,...,...,...
60664,PNH00002030401,2019,PA,19067,1906.0,1
60665,PNH00002031363,2019,PA,16667,8632.0,0
60666,PNH00002031395,2018,PA,17015,9049.0,0
60667,PNH00002031395,2019,PA,17015,9049.0,0


In [112]:
idyr_t.groupby(['year','no_hit'])['pol_id'].count()

year  no_hit
2016  0          6995
      1           726
2017  0          8448
      1           898
2018  0         13678
      1          1482
2019  0         20523
      1          1974
2020  0          5585
      1           360
Name: pol_id, dtype: int64

In [113]:
idyr_X_test = idyr_t.drop('no_hit', axis=1)
idyr_y_test = idyr_t[['no_hit']]

In [114]:
idyr_X_test.groupby('year')['pol_id'].count()

year
2016     7721
2017     9346
2018    15160
2019    22497
2020     5945
Name: pol_id, dtype: int64

In [115]:
idyr_X_test.groupby('STATE')['pol_id'].count()

STATE
CT     4669
MA    31893
NH     3146
NJ    15139
NY      310
PA     5512
Name: pol_id, dtype: int64

In [116]:
idyr_y_test.value_counts()

no_hit
0         55229
1          5440
dtype: int64

In [117]:
idyr_t.to_csv('idyr_test.csv')

# Policy id + year (out of sample)

In [118]:
out_id = p_out[['pol_id']]

In [119]:
idyr_out = pd.merge(df, out_id, how='inner', on='pol_id')
idyr_out

,pol_id,year,STATE,ZIP5,ZIP4,no_hit
0,BHD00001001034,2015,MA,2745,5672.0,0
1,BHD00001001080,2015,MA,1850,2164.0,0
2,BHD00001001157,2015,MA,2122,2204.0,0
3,BHD00001001189,2015,MA,1571,3437.0,1
4,BHD00001001232,2015,MA,2151,5005.0,0
...,...,...,...,...,...,...
98243,PNH00002031667,2020,PA,19145,1620.0,0
98244,PNH00002031669,2020,PA,15136,3130.0,1
98245,PNH00002031671,2020,PA,19126,3536.0,0
98246,PNH00002031672,2020,PA,19148,1731.0,0


In [120]:
idyr_out.to_csv('idyr_outsample.csv')

In [121]:
idyr_out_X = idyr_out.drop('no_hit', axis=1)
idyr_out_y = idyr_out[['no_hit']]

In [122]:
idyr_out_X.groupby('year')['pol_id'].count()

year
2011     3395
2012     4084
2013     6421
2014     9307
2015    17002
2016     9935
2017     8346
2018     6958
2019     5772
2020    27028
Name: pol_id, dtype: int64

In [123]:
idyr_out_X.groupby('STATE')['pol_id'].count()

STATE
CT    32872
MA    13216
NH    35575
NJ     6705
NY     5760
PA     4120
Name: pol_id, dtype: int64

In [124]:
idyr_out_y.value_counts()

no_hit
0         87053
1         11195
dtype: int64

In [125]:
idyr_out.groupby(['year','no_hit'])['pol_id'].count()

year  no_hit
2011  0          3122
      1           273
2012  0          3757
      1           327
2013  0          5834
      1           587
2014  0          8610
      1           697
2015  0         15916
      1          1086
2016  0          9503
      1           432
2017  0          7997
      1           349
2018  0          6704
      1           254
2019  0          5584
      1           188
2020  0         20026
      1          7002
Name: pol_id, dtype: int64